In [1]:
import pynini
import pywrapfst as fst

In [2]:
def create_lists(g_far_path, p_far_path):
    g_far_reader = pynini.Far(g_far_path, mode = 'r') 
    p_far_reader = pynini.Far(p_far_path, mode = 'r') 
    g_container = []
    p_container = []
    while not g_far_reader.done():   
        g_fsa = g_far_reader.get_fst()
        g_labels_extract = g_fsa.paths().ilabels()
        g_container.append(g_labels_extract)
        g_far_reader.next()
    while not p_far_reader.done():   
        p_fsa = p_far_reader.get_fst()
        p_labels_extract = p_fsa.paths().ilabels()
        p_container.append(p_labels_extract)
        p_far_reader.next()
    
    g_far_reader.reset()
    p_far_reader.reset()
    return g_container, p_container


def make_lattice(container: list):
    lattice = pynini.Fst()
    starting_state = lattice.add_state()
    lattice.set_start(starting_state)
    for labels in container:
        state = starting_state
        for label in labels:
            nextstate = lattice.add_state()
            lattice.add_arc(state, pynini.Arc(label, label, pynini.Weight.One(lattice.weight_type()), nextstate)) 
            state = nextstate
        g_lattice = lattice.set_final(state)
    assert lattice.verify(), "Invalid lattice"
    return lattice.closure()



In [10]:
def main(g_far_path, p_far_path):
    g_container, p_container = create_lists(g_far_path, p_far_path)   
    g_lattice = make_lattice(g_container)
    p_lattice = make_lattice(p_container)
    pynini.transducer(g_lattice, p_lattice, weight=None, arc_type=None,
           input_token_type="byte", output_token_type="byte",
           attach_input_symbols=True, attach_output_symbols=True)
    

In [ ]:
main('h_g.far', 'g_p.far')